In [1]:
import os
import pandas as pd
from tqdm import tqdm
import re

In [2]:
# Mount google drive (for Colab only)
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive',force_remount=True)
    base_folder = '/content/drive/My Drive/unibo/NLP_project/BarneyBot'
else:
    base_folder = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
    
in_folder = os.path.join(base_folder, "in")
if not os.path.exists(in_folder):
    os.makedirs(in_folder)

# Preprocessing

In [3]:
# Open the dataset documents and store their data into a DataFrame
def load_himym_dataset():
    episodes_folder = os.path.join(base_folder, "Datasets", "Sources", "HIMYM", "Episodes")
    dataframe_rows = []
    # Get number of documents and their names
    documents_n = len(os.listdir(episodes_folder))
    documents_names = os.listdir(episodes_folder)

    # Loop over documents
    for i in tqdm(range(documents_n)):
        filename = documents_names[i]
        episode_index = filename[:-4]
        # Open document
        with open(os.path.join(episodes_folder, filename)) as file:
            # Loop over lines (= words)
            for line in file.readlines():
                    dataframe_row = {
                        "episode": episode_index,
                        "line": line,
                    }
                    dataframe_rows.append(dataframe_row)
    # Build the dataframe from the words
    df = pd.DataFrame(dataframe_rows)
    return df

In [4]:
# Execute creation of dataset
himym_df = load_himym_dataset()
himym_df.head()
himym_df.count()

100%|███████████████████████████████████████████████████████████████████████████████| 208/208 [00:01<00:00, 158.89it/s]


episode    71983
line       71983
dtype: int64

In [5]:
def process_himym_dataset(df):
    df = df[~df['line'].str.startswith("[")]
    df = df[~df['line'].str.startswith("(")]
    df['line'] = df['line'].str.strip()
    df['line'] = df['line'].str.replace(r"\(.*\)","")
    df['line'] = df['line'].str.replace(r"[\/(){}\[\]\|@_#]|\\t|\\n"," ")
    df['line'] = df['line'].str.replace(r"[^.\',;:?!0-9a-zA-Z \-]","")
    df = df[~df['line'].isnull()]
    df[['character', 'line']] = df['line'].str.split(":", 1, expand=True)
    df = df.dropna()
    df['line'] = df['line'].str.strip()
    df['line'] = df['line'][df['line'].str.len() >= 2]
    df = df[~df['line'].isnull()]
    df = df.dropna()
    df = df.reset_index(drop=True)
    return df

In [6]:
himym_df = process_himym_dataset(himym_df)
print(len(himym_df))

<ipython-input-5-a2cc5ebe8637>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['line'] = df['line'].str.replace(r"\(.*\)","")
<ipython-input-5-a2cc5ebe8637>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df['line'] = df['line'].str.replace(r"[\/(){}\[\]\|@_#]|\\t|\\n"," ")
<ipython-input-5-a2cc5ebe8637>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df['line'] = df['line'].str.replace(r"[^.\',;:?!0-9a-zA-Z \-]","")


31776


In [7]:
himym_df.head()

,episode,line,character
0,01x01,"Kids, I'm going to tell you an incredible stor...",Narrator
1,01x01,Are we being punished for something?,Son
2,01x01,No,Narrator
3,01x01,"Yeah, is this going to take a while?",Daughter
4,01x01,"Yes. Twenty-five years ago, before I was dad,...",Narrator


In [8]:
barney_names = [c for c in himym_df['character'] if 'Barney' in c]
print(set(barney_names), len(barney_names))

{'Marshall and Barney', 'Barney, crying', 'Both Ted and Barney, at the same time', 'Ted, Barney and Robin', 'Barney, joining them', 'Lily, holding Barney', 'Marshall, Lily and Barney', 'Ted and Barney', 'Barney by nightdress', 'Barney and James', 'Barney and Robin', "Okay, but, Barney, just remember, you'll also be seeing a lot of this", 'Lily, Marshall, Barney', 'Stacy, Barney', 'Barney, Ted and Robin', 'Marshall, Barney', 'Ted, from seeing Barney', 'Robin and Barney', 'Barney lookalike', 'Barney conscience', 'Little Barney', 'Marshall, on the phone with Barney', 'Barney, with a funny accent', 'Barney  Robin', 'Barney, quick question', "Barney's mom", "At Casa a pezzi. Barney is playing the piano.Ted's father", 'Barney, whispering', 'Barney, laughing', 'Barney, singsongy', 'Barney, his voice breaking', 'Marshall, Lily, Barney', 'Barney, turning', 'Barney, entering', 'Ted  Barney', 'Barney  Ted', "Anna's apartment. Anna's reading the letter Barney left.Barney's voice", 'Marshall, to th

In [9]:
barney_names = set(barney_names) - set(["Barney's Secretary", 'Marshall to Barney', "Barney's mom", 'Ted, from seeing Barney', 'Lily, holding Barney', 'Marshall, on the phone with Barney', "At Casa a pezzi. Barney is playing the piano.Ted's father", 'Marshall, to the girl Barney is talking to'])

In [10]:
himym_df['character'] = himym_df['character'].apply(lambda x: 'Barney' if 'Barney' in x else x)

In [11]:
himym_df['character'].unique()

array(['Narrator', 'Son', 'Daughter', ...,
       'Excuse me. It even has my initials on it right here',
       'Those are my initials',
       'Terribly Mistaken, because this umbrella has always belonged T.M.'],
      dtype=object)

In [12]:
himym_path = os.path.join(base_folder, "Datasets", "Characters", "Barney")
if not os.path.exists(himym_path):
    os.makedirs(himym_path)
himym_df.to_csv(os.path.join(himym_path, "HIMYM_preprocessed.csv"), index=False)

In [13]:
# NOTE: May consider feeding one sentence and one Sheldon reply or multiple sentences encoded with one Sheldon reply
def get_barney(himym_df, level=2):
    dataframe_rows = []
    idxs_barney = himym_df[himym_df['character'] == 'Barney'].index
    dataframe_rows = []
    for i in idxs_barney:
        l = []
        l.append(himym_df['line'][i])
        for j in range(0,level):
            line = max(i-j-1,0)
            l.append(himym_df['line'][line])
        dataframe_rows.append(l)
    df = pd.DataFrame(dataframe_rows, columns=['response', 'context', 'context/0'])
    return df

barney_df = get_barney(himym_df)

In [14]:
barney_df.head()

,response,context,context/0
0,"hey, so you know how I've always had a thing f...",What was I doing? Your Uncle Marshall was taki...,"Yeah, what are you doing tonight?"
1,"Okay, meet me at the bar in fifteen minutes, a...","Hey, you wanna do something tonight?","hey, so you know how I've always had a thing f..."
2,Where's your suit!? Just once when I say suit ...,Hey.,"Okay, meet me at the bar in fifteen minutes, a..."
3,It was a blazer!,I did that one time.,Where's your suit!? Just once when I say suit ...
4,I see what this is about. Have you forgotten w...,"You know, ever since college it's been Marshal...",It was a blazer!


In [15]:
len(barney_df)

5142

In [16]:
barney_path = os.path.join(base_folder, "Datasets", "Characters", "Barney")
if not os.path.exists(barney_path):
    os.makedirs(barney_path)
barney_df.to_csv(os.path.join(barney_path, "Barney.csv"), index=False)